# Homework 3

## FINM 37500: Fixed Income Derivatives

### Mark Hendricks

#### Winter 2024

# 1. Modeling the Volatility Smile

## Swaption Vol Data

The file `data/swaption_vol_data_2024-02-20.xlsx` has market data on the implied volatility skews for swaptions. Note that it has several columns:
* `expry`: expiration of the swaption
* `tenor`: tenor of the underlying swap
* `model`: the model by which the volatility is quoted. (All are Black.)
* `-200`, `-100`, etc.: The strike listed as difference from ATM strike (bps). Note that ATM is considered to be the **forward swapa rate** which you can calculate.

Your data: ywill use a single row of this data for the `1x4` swaption.
* date: `2024-02-20`
* expiration: 1yr
* tenor: 4yrs

## Rate Data

The file `data/cap_quotes_2024-02-20.xlsx` gives 
* SOFR swap rates, 
* their associated discount factors
* their associated forward interest rates.

You will not need the cap data (flat or forward vols) for this problem.
* This cap data would be helpful in calibrating a binomial tree, but this problem focuses on Black's formula and SABR.

## The Swaption

Consider the following swaption with the following features:
* underlying is a fixed-for-floating (SOFR) swap
* the underlying swap has **quarterly** payment frequency
* this is a **payer** swaption, which gives the holder the option to **pay** the fixed swap rate and receive SOFR.

## 1.1
Calculate the (relevant) forward swap rate. That is, the one-year forward 4-year swap rate.

## 1.2
Price the swaptions at the quoted implied volatilites and corresponding strikes, all using the just-calculated forward swap rate as the underlying.

## 1.3
To consider how the expiration and tenor matter, calculate the prices of a few other swaptions for comparison. 
* No need to get other implied vol quotes--just use the ATM implied vol you have for the 1x2 above. (Here we are just interested in how Black's formula changes with changes in tenor and expiration.
* No need to calculate for all the strikes--just do the ATM strike.

Alternate swaptions
* The 3mo x 4yr swaption
* The 2yr x 4yr swaption
* the 1yr x 2yr swaption

Report these values and compare them to the price of the `1y x 4y` swaption.

In [1]:
import pandas as pd
import numpy as np

from binomial import *
from ratecurves import *
from volskew import *
from ficcvol import *

In [2]:
vol = pd.read_excel('swaption_vol_data.xlsx')
vol

,reference,instrument,model,date,expiration,tenor,-200,-100,-50,-25,0,25,50,100,200
0,SOFR,swaption,black,2024-02-20,1,4,54.54,40.37,35.94,34.23,32.83,31.71,30.86,29.83,29.54


In [3]:
cap_curves = pd.read_excel('cap_curves_2024-02-20.xlsx')
cap_curves.set_index('tenor', inplace=True)
cap_curves

,swap rates,spot rates,discounts,forwards,flat vols,fwd vols
tenor,,,,,,
0.25,0.052211,0.052211,0.987115,NaN,NaN,NaN
0.50,0.051540,0.051535,0.974722,0.050860,0.166025,0.166025
0.75,0.050506,0.050490,0.963069,0.048400,0.191290,0.210648
1.00,0.049284,0.049250,0.952230,0.045531,0.216554,0.254312
1.25,0.047631,0.047565,0.942608,0.040831,0.260043,0.361247
1.50,0.046235,0.046141,0.933499,0.039030,0.292615,0.380930
1.75,0.045059,0.044939,0.924774,0.037738,0.315878,0.388953
2.00,0.044133,0.043994,0.916212,0.037382,0.331443,0.386643
2.25,0.043173,0.043011,0.908230,0.035151,0.340919,0.376247


## 1.1

In [4]:
start = 1
end = start + 4

forward_swap_rate = calc_fwdswaprate(cap_curves['discounts'], start, end, freqswap=4)
forward_swap_rate

0.03672212061985555

## 1.2

In [5]:
strikerange = np.array(vol.columns[-9:].tolist())
vols = vol[strikerange] / 100
strikes = forward_swap_rate + strikerange/100/100

period_fwd = cap_curves.index.get_loc(start)
period_swap = cap_curves.index.get_loc(end) + 1

discount = cap_curves['discounts'].iloc[period_fwd+1 : period_swap].sum() / 4

quotes = vols.copy()
quotes.loc['price'] = 100 * blacks_formula(start, vols, strikes, forward_swap_rate, discount, isCall=True)[0]
quotes.loc['strike'] = strikes
quotes = quotes.rename(index={0: 'implied vol'})
quotes = quotes.loc[['strike', 'implied vol', 'price']]
quotes

,-200,-100,-50,-25,0,25,50,100,200
strike,0.016722,0.026722,0.031722,0.034222,0.036722,0.039222,0.041722,0.046722,0.056722
implied vol,0.545400,0.403700,0.359400,0.342300,0.328300,0.317100,0.308600,0.298300,0.295400
price,7.203088,4.065315,2.739848,2.174850,1.687383,1.281199,0.955570,0.513281,0.147265


## 1.3

In [6]:
expiries = [.25,2,1]
tenors = [4,4,2]
fwdswaps = np.full(len(expiries), np.nan)

quotes_alt = pd.DataFrame(columns=['expiry', 'tenor', 'price'])

idstrikeATM = np.where(strikerange==0)[0][0]
strikeATM = strikes[idstrikeATM]
volATM = vols.iloc[0,idstrikeATM]

for i in range(len(fwdswaps)):
    fwdswaps[i] = calc_fwdswaprate(cap_curves['discounts'], expiries[i], expiries[i]+tenors[i], freqswap=4)
    
    period0 = cap_curves.index.get_loc(expiries[i])
    period1 = cap_curves.index.get_loc(expiries[i]+tenors[i])+1
    
    discount = cap_curves['discounts'].iloc[period0+1 : period1].sum() / 4
    
    quotes_alt.loc[i,['expiry', 'tenor']] = [expiries[i], tenors[i]]
    quotes_alt.loc[i, 'price'] = 100 * blacks_formula(expiries[i], volATM, strikeATM, forward_swap_rate, discount, isCall=True)
    
quotes_alt

,expiry,tenor,price
0,0.25,4,0.870475
1,2,4,2.291364
2,1,2,0.874004


***

# 2. Pricing w/ BDT

Use the data in `cap_curves_2024-02-20.xlsx`.

## 2.1

Calibrate the BDT Tree
* theta to fit the term structure discounts.
* sigma to fit the fwd vols from the cap data.

Report the rate tree through $T=5$. Report trees for rates compounded
* continuously
* annually

In [7]:
sig = cap_curves['fwd vols']
sig.iloc[0] = sig.iloc[1]
sig

tenor
0.25     0.166025
0.50     0.166025
0.75     0.210648
1.00     0.254312
1.25     0.361247
1.50     0.380930
1.75     0.388953
2.00     0.386643
2.25     0.376247
2.50     0.363764
2.75     0.353513
3.00     0.349207
3.25     0.350339
3.50     0.351556
3.75     0.349233
4.00     0.339501
4.25     0.322525
4.50     0.309744
4.75     0.306238
5.00     0.311414
5.25     0.324078
5.50     0.332488
5.75     0.331264
6.00     0.317965
6.25     0.294203
6.50     0.273998
6.75     0.262325
7.00     0.260895
7.25     0.268693
7.50     0.275091
7.75     0.276397
8.00     0.271592
8.25     0.261350
8.50     0.251699
8.75     0.244536
9.00     0.240251
9.25     0.239254
9.50     0.241978
9.75     0.248893
10.00    0.260524
Name: fwd vols, dtype: float64

In [8]:
disc = cap_curves['discounts']
theta_params, rate_tree = estimate_theta(sig, disc)

In [9]:
pd.DataFrame(theta_params).T

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,...,7.50,7.75,8.00,8.25,8.50,8.75,9.00,9.25,9.50,9.75
theta,NaN,-0.117846,-0.262526,-0.359865,-0.720504,-0.459257,-0.384915,-0.244081,-0.396789,-0.033089,...,0.125541,0.134603,0.153908,0.169295,0.178789,0.182346,0.179249,0.168698,0.149782,0.121431


In [10]:
# Continuously
rate_tree = rate_tree[[col for col in rate_tree.columns if col <= 5]]
rate_tree = rate_tree.dropna(how='all')
rate_tree.round(3)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,...,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75,5.00
state,,,,,,,,,,,,,,,,,,,,,
0,0.052,0.055,0.057,0.059,0.059,0.064,0.070,0.080,0.088,0.104,...,0.150,0.171,0.200,0.247,0.298,0.343,0.406,0.500,0.608,0.721
1,NaN,0.046,0.048,0.050,0.050,0.054,0.060,0.068,0.074,0.088,...,0.127,0.145,0.170,0.210,0.252,0.291,0.344,0.424,0.515,0.611
2,NaN,NaN,0.039,0.041,0.041,0.044,0.048,0.055,0.060,0.072,...,0.103,0.117,0.137,0.170,0.204,0.236,0.279,0.343,0.417,0.495
3,NaN,NaN,NaN,0.031,0.031,0.034,0.037,0.043,0.047,0.056,...,0.080,0.091,0.107,0.132,0.158,0.183,0.216,0.266,0.323,0.384
4,NaN,NaN,NaN,NaN,0.022,0.024,0.026,0.030,0.033,0.039,...,0.056,0.063,0.074,0.092,0.110,0.127,0.151,0.185,0.225,0.267
5,NaN,NaN,NaN,NaN,NaN,0.016,0.018,0.020,0.022,0.026,...,0.038,0.043,0.051,0.063,0.075,0.087,0.103,0.127,0.154,0.183
6,NaN,NaN,NaN,NaN,NaN,NaN,0.012,0.014,0.015,0.018,...,0.026,0.029,0.034,0.042,0.051,0.059,0.070,0.086,0.104,0.124
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009,0.010,0.012,...,0.018,0.020,0.023,0.029,0.035,0.040,0.047,0.058,0.071,0.084
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007,0.008,...,0.012,0.014,0.016,0.020,0.024,0.027,0.033,0.040,0.049,0.058


In [11]:
# Annually
rate_tree_annual = np.exp(rate_tree) - 1
rate_tree_annual.round(4)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,...,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75,5.00
state,,,,,,,,,,,,,,,,,,,,,
0,0.0532,0.0563,0.0586,0.0609,0.0609,0.0659,0.0729,0.0836,0.0918,0.1101,...,0.1621,0.1864,0.2217,0.2807,0.3465,0.4095,0.5012,0.6488,0.8366,1.0569
1,NaN,0.0474,0.0494,0.0513,0.0514,0.0555,0.0614,0.0704,0.0772,0.0925,...,0.1357,0.1558,0.1849,0.2331,0.2866,0.3374,0.4107,0.5274,0.6735,0.8420
2,NaN,NaN,0.0398,0.0414,0.0414,0.0447,0.0495,0.0567,0.0621,0.0743,...,0.1086,0.1245,0.1473,0.1850,0.2265,0.2656,0.3215,0.4093,0.5176,0.6402
3,NaN,NaN,NaN,0.0319,0.0320,0.0345,0.0381,0.0437,0.0478,0.0571,...,0.0832,0.0952,0.1124,0.1407,0.1715,0.2004,0.2413,0.3048,0.3819,0.4677
4,NaN,NaN,NaN,NaN,0.0222,0.0239,0.0264,0.0302,0.0331,0.0395,...,0.0573,0.0654,0.0771,0.0961,0.1166,0.1357,0.1626,0.2037,0.2528,0.3065
5,NaN,NaN,NaN,NaN,NaN,0.0163,0.0180,0.0206,0.0225,0.0268,...,0.0388,0.0443,0.0520,0.0647,0.0783,0.0908,0.1084,0.1350,0.1665,0.2004
6,NaN,NaN,NaN,NaN,NaN,NaN,0.0122,0.0139,0.0152,0.0181,...,0.0261,0.0298,0.0350,0.0434,0.0524,0.0607,0.0722,0.0896,0.1100,0.1318
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0094,0.0103,0.0123,...,0.0177,0.0201,0.0236,0.0293,0.0353,0.0408,0.0485,0.0601,0.0735,0.0877
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0071,0.0084,...,0.0121,0.0138,0.0162,0.0200,0.0241,0.0279,0.0331,0.0408,0.0499,0.0594


## 2.2

Use a tree to price a vanilla fixed-rate, 5-year bond with coupon rate equal to the forward swap rate calculated in problem `1.1.`

In [12]:
FACE = 100
T = 5
compound = 4
cpn = forward_swap_rate
cpn_freq = 4

In [13]:
payoff = lambda r: payoff_bond(r, 1/compound, facevalue = FACE * (1 + cpn / cpn_freq))
cash_flow_tree = construct_bond_cftree(T, compound, cpn, cpn_freq=cpn_freq)
bond_tree = bintree_pricing(payoff=payoff, ratetree=rate_tree.iloc[:int(T * compound),:int(T * compound)], cftree=cash_flow_tree)
bond_tree.round(4)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,98.7824,96.6642,94.4413,92.0871,89.5921,86.9021,84.1256,81.3138,78.5532,75.8129,73.3055,71.1427,69.4533,68.2088,67.6276,68.1191,69.8804,73.0306,78.2120,86.6895
1,NaN,101.6432,99.7142,97.6675,95.4874,93.1145,90.6360,88.0871,85.5366,82.9433,80.4948,78.2881,76.4411,74.9272,73.9576,73.9288,75.0414,77.4176,81.6270,88.7285
2,NaN,NaN,104.1057,102.3441,100.4714,98.4305,96.2890,94.0698,91.8233,89.5034,87.2649,85.1853,83.3597,81.7547,80.5524,80.1051,80.5949,82.1447,85.2817,90.9247
3,NaN,NaN,NaN,106.0748,104.4663,102.7248,100.9014,99.0106,97.0901,95.0933,93.1412,91.2925,89.6220,88.0901,86.8356,86.1468,86.1677,86.9852,89.0288,93.0789
4,NaN,NaN,NaN,NaN,107.5216,106.0216,104.4624,102.8536,101.2217,99.5314,97.8695,96.2783,94.8120,93.4405,92.2648,91.4979,91.2450,91.5654,92.7653,95.3891
5,NaN,NaN,NaN,NaN,NaN,108.3673,107.0019,105.6020,104.1864,102.7298,101.2933,99.9066,98.6082,97.3761,96.2814,95.4754,95.0313,94.9880,95.5566,97.1069
6,NaN,NaN,NaN,NaN,NaN,NaN,108.7737,107.5212,106.2585,104.9684,103.6934,102.4545,101.2787,100.1509,99.1202,98.2917,97.7151,97.4152,97.5341,98.3190
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108.8480,107.6904,106.5147,105.3503,104.2123,103.1196,102.0623,101.0745,100.2287,99.5585,99.0796,98.8870,99.1451
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108.6799,107.5820,106.4925,105.4219,104.3838,103.3715,102.4093,101.5475,100.8092,100.2043,99.7972,99.6976


In [14]:
bond_tree[0][0].round(2)

98.78

## 2.3

We will calculate the binomial tree for the 5-year swap, but here we do so by valuing the swap as...

$$\text{payer swap} = \text{floating rate note} - \text{fixed-rate bond}$$

Recall for the Floating-Rate Note:
* It has par value of 100 at each reset date.
* Every node is a reset date given the assumptions of the swap timing.

Report the tree for the 5-year swap.

In [15]:
swap_tree = 100 - bond_tree
swap_tree

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,1.217632,3.335809,5.558689,7.912866,10.407860,13.097858,15.874401,18.686214,21.446778,24.187076,26.694490,28.857323,30.546699,31.791240,32.372377,31.880900,30.119641,26.969433,21.787994,13.310502
1,NaN,-1.643221,0.285778,2.332497,4.512569,6.885497,9.364037,11.912930,14.463449,17.056662,19.505234,21.711924,23.558940,25.072782,26.042388,26.071234,24.958627,22.582447,18.373034,11.271489
2,NaN,NaN,-4.105667,-2.344113,-0.471436,1.569486,3.710952,5.930190,8.176713,10.496597,12.735116,14.814670,16.640276,18.245265,19.447635,19.894874,19.405095,17.855262,14.718285,9.075285
3,NaN,NaN,NaN,-6.074835,-4.466336,-2.724765,-0.901353,0.989438,2.909907,4.906688,6.858799,8.707469,10.378022,11.909910,13.164414,13.853174,13.832280,13.014783,10.971198,6.921123
4,NaN,NaN,NaN,NaN,-7.521593,-6.021599,-4.462396,-2.853616,-1.221742,0.468577,2.130483,3.721733,5.187963,6.559486,7.735214,8.502063,8.754990,8.434566,7.234710,4.610935
5,NaN,NaN,NaN,NaN,NaN,-8.367281,-7.001858,-5.602038,-4.186391,-2.729823,-1.293290,0.093415,1.391815,2.623912,3.718567,4.524629,4.968743,5.011977,4.443365,2.893086
6,NaN,NaN,NaN,NaN,NaN,NaN,-8.773703,-7.521235,-6.258491,-4.968413,-3.693432,-2.454534,-1.278737,-0.150858,0.879769,1.708313,2.284896,2.584839,2.465871,1.680991
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.848033,-7.690422,-6.514667,-5.350334,-4.212304,-3.119637,-2.062255,-1.074490,-0.228675,0.441471,0.920442,1.112958,0.854939
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.679919,-7.582045,-6.492507,-5.421900,-4.383763,-3.371510,-2.409324,-1.547518,-0.809194,-0.204308,0.202838,0.302365


## 2.4



Report the binomial tree for the one-year swaption on a 4-year swap with **european** exercise.
* At expiration, the swap tree from 2.3 will have 4 years left, as desired for pricing the 1y-4y swaption.

In [16]:
payoff_swaption = lambda p: np.maximum(p, 0)

T = 1
tsteps = int(T * compound) + 1

swaption_tree = bintree_pricing(payoff_swaption, rate_tree.iloc[:tsteps,:tsteps], undertree=swap_tree.iloc[:tsteps,:tsteps])
swaption_tree.round(3)

time,0.00,0.25,0.50,0.75,1.00
state,,,,,
0,1.686,2.872,4.722,7.351,10.408
1,NaN,0.544,1.101,2.228,4.513
2,NaN,NaN,0.000,0.000,0.000
3,NaN,NaN,NaN,0.000,0.000
4,NaN,NaN,NaN,NaN,0.000


## 2.5

Compare the pricing of the 1y4y swaption from Black's formula in Section 1 vs the binomial tree.

In section 1, our price was roughly 1.69. Here, our price is about 1.69 as well.

## 2.6

Reprice the swaption using the BDT tree, but this time assuming it is **american**-style exercise.

In [17]:
bintree_pricing(payoff_swaption, rate_tree.iloc[:tsteps,:tsteps], undertree=swap_tree.iloc[:tsteps,:tsteps], style='american').round(3)

time,0.00,0.25,0.50,0.75,1.00
state,,,,,
0,1.927,3.336,5.559,7.913,10.408
1,NaN,0.569,1.152,2.332,4.513
2,NaN,NaN,0.000,0.000,0.000
3,NaN,NaN,NaN,0.000,0.000
4,NaN,NaN,NaN,NaN,0.000


***

# 3. Midcurve Swaptions

## 3.1 

Use the BDT tree from section 2 to price a **european** midcurve swaption 1y$\rightarrow$2y$\rightarrow$2y.

In [18]:
FACE = 100
T = 5
compound = 4
cpn = forward_swap_rate
cpn_freq = 4

In [19]:
payoff = lambda r: payoff_bond(r, 1/compound, facevalue = FACE * (1 + cpn / cpn_freq))
cash_flow_tree = construct_bond_cftree(T, compound, cpn, cpn_freq=cpn_freq)
cash_flow_tree.iloc[:, :12] = 0
delayed_swap_tree = bintree_pricing(payoff=payoff, ratetree=rate_tree.iloc[:int(T * compound),:int(T * compound)], cftree=cash_flow_tree)
delayed_swap_tree.iloc[:, 12:] = 100 - delayed_swap_tree.iloc[:, 12:]
delayed_swap_tree.iloc[:, :12] = 0
delayed_swap_tree

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.546699,31.791240,32.372377,31.880900,30.119641,26.969433,21.787994,13.310502
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.558940,25.072782,26.042388,26.071234,24.958627,22.582447,18.373034,11.271489
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.640276,18.245265,19.447635,19.894874,19.405095,17.855262,14.718285,9.075285
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.378022,11.909910,13.164414,13.853174,13.832280,13.014783,10.971198,6.921123
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.187963,6.559486,7.735214,8.502063,8.754990,8.434566,7.234710,4.610935
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.391815,2.623912,3.718567,4.524629,4.968743,5.011977,4.443365,2.893086
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.278737,-0.150858,0.879769,1.708313,2.284896,2.584839,2.465871,1.680991
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.119637,-2.062255,-1.074490,-0.228675,0.441471,0.920442,1.112958,0.854939
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.383763,-3.371510,-2.409324,-1.547518,-0.809194,-0.204308,0.202838,0.302365


Set payoff to be 0 before t=3. To do this, we observe that everything before t=3 is above 50 and everything starting t=3 and after is below 50.

In [20]:
payoff = lambda p: p

delayed_swap_tree = bintree_pricing(payoff=payoff, ratetree=rate_tree.iloc[:13,:13], undertree=delayed_swap_tree.iloc[:13,:13])
delayed_swap_tree

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00
state,,,,,,,,,,,,,
0,-0.129221,0.988881,2.290994,3.802076,5.546758,7.543229,9.816798,12.387915,15.279422,18.487914,22.062377,26.055695,30.546699
1,NaN,-1.250696,-0.285986,0.845606,2.170580,3.715495,5.512130,7.594097,9.998895,12.748980,15.891468,19.470303,23.558940
2,NaN,NaN,-2.244564,-1.424517,-0.458070,0.680364,2.019897,3.595648,5.449812,7.624017,10.176604,13.165493,16.640276
3,NaN,NaN,NaN,-3.108674,-2.419994,-1.605843,-0.644202,0.493200,1.841241,3.440975,5.347024,7.629018,10.378022
4,NaN,NaN,NaN,NaN,-3.846423,-3.272364,-2.594846,-1.793718,-0.844247,0.284759,1.631185,3.244400,5.187963
5,NaN,NaN,NaN,NaN,NaN,-4.462758,-3.988685,-3.429929,-2.769994,-1.987047,-1.056128,0.056004,1.391815
6,NaN,NaN,NaN,NaN,NaN,NaN,-4.972952,-4.583066,-4.124841,-3.583823,-2.944333,-2.185054,-1.278737
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.392919,-5.072943,-4.696999,-4.255511,-3.735304,-3.119637
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.738182,-5.474889,-5.167133,-4.806807,-4.383763


In [21]:
payoff_swaption = lambda p: np.maximum(p, 0)

T = 1
tsteps = int(T * compound) + 1

midcurve_swaption_tree = bintree_pricing(payoff_swaption, rate_tree.iloc[:tsteps,:tsteps], undertree=delayed_swap_tree.iloc[:tsteps,:tsteps])
midcurve_swaption_tree.round(3)

time,0.00,0.25,0.50,0.75,1.00
state,,,,,
0,0.843,1.446,2.402,3.802,5.547
1,NaN,0.262,0.529,1.072,2.171
2,NaN,NaN,0.000,0.000,0.000
3,NaN,NaN,NaN,0.000,0.000
4,NaN,NaN,NaN,NaN,0.000


## 3.2

Price the **american** midcurve swaption 1y$\rightarrow$2y$\rightarrow$2y.

In [22]:
bintree_pricing(payoff_swaption, rate_tree.iloc[:tsteps,:tsteps], undertree=delayed_swap_tree.iloc[:tsteps,:tsteps], style='american').round(3)

time,0.00,0.25,0.50,0.75,1.00
state,,,,,
0,0.843,1.446,2.402,3.802,5.547
1,NaN,0.262,0.529,1.072,2.171
2,NaN,NaN,0.000,0.000,0.000
3,NaN,NaN,NaN,0.000,0.000
4,NaN,NaN,NaN,NaN,0.000


***